In [1]:
# -*- coding: utf-8 -*- 

import re 
from tqdm import tqdm 
import time
from datetime import datetime
import sqlite3
import sys
import os
import pandas as pd
import unify

In [7]:
def connect(file_path, primary, columns):
    con = sqlite3.connect(file_path)
    cur = con.cursor()
    cols = ", ".join([c + ' Varchar' for c in columns]) 
    cur.execute("create table meta ("+primary+" Varchar PRIMARY KEY, "+cols+" )")
    cur.execute("CREATE INDEX log on meta (id);")
    cur.execute("create table plain_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL);")
    cur.execute("create table tagged_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL );")
    con.commit()
    return con, cur

In [4]:
workdir = r'/home/tari/Загрузки/taiga/Subtitles_norm'
filename = 'Subtitles.db'
file_path = os.path.join(workdir, filename)
metatablepath = os.path.join(workdir,'newmetadata.csv')
tagged = os.path.join(workdir,'texts_tagged')
plain = os.path.join(workdir,'texts')

meta = pd.read_csv(metatablepath, sep='\t', encoding='utf8')
meta = meta.fillna('')
meta.head()

,id,title,languages,filepath
0,0,10 Things I Hate About You - 1x01 - Pilot.HDTV...,en,10 Things I Hate About You - 1x01 - Pilot.HDTV...
1,1,10 Things I Hate About You - 1x01 - Pilot.HDTV...,i,10 Things I Hate About You - 1x01 - Pilot.HDTV...
2,2,10 Things I Hate About You - 1x01 - Pilot.HDTV...,ru,10 Things I Hate About You - 1x01 - Pilot.HDTV...
3,3,10 Things I Hate About You - 1x02 - I Want You...,en,10 Things I Hate About You - 1x02 - I Want You...
4,4,10 Things I Hate About You - 1x02 - I Want You...,ru,10 Things I Hate About You - 1x02 - I Want You...


In [5]:
meta.columns

Index(['id', 'title', 'languages', 'filepath'], dtype='object')

In [9]:
if not os.path.exists(filename):
    con, cur = connect(filename, meta.columns[1], [meta.columns[0]]+list(meta.columns[2:]))
else:
    con = sqlite3.connect(filename, meta.columns[1], [meta.columns[0]]+list(meta.columns[2:]))
    cur = con.cursor()

In [11]:
meta.iloc[7].to_dict()

{'filepath': "10 Things I Hate About You - 1x03 - Won't Get Fooled Again.HDTV.FQM.ru.txt",
 'id': 7,
 'languages': 'ru',
 'title': "10 Things I Hate About You - 1x03 - Won't Get Fooled Again.HDTV.FQM.ru.srt"}

In [12]:
m = meta.iloc[7].to_dict()
m['filepath'].split(' - ')[0]


'10 Things I Hate About You'

In [ ]:
for i in tqdm(range(len(meta))):
    values = meta.iloc[i].to_dict()
    folder = values['filepath'].split(' - ')[0]

    values['id'] = str(values['id'])
    columns = ', '.join(values.keys())
    
    #print(list(values.values()))
    placeholders = ', '.join('?' * len(values))
    sql = 'INSERT INTO meta ({}) VALUES ({})'.format(columns, placeholders)
    #print(sql)
    cur.execute(sql, list(values.values()))
    
    valuest = {'id': values['id'], 'text': unify.open_text(os.path.join(os.path.join(plain,folder), str(values['filepath'])+".txt"))}
    
    columns = ', '.join(valuest.keys())
    placeholders = ', '.join('?' * len(valuest))
    sql2 = 'INSERT INTO plain_texts ({}) VALUES ({})'.format(columns, placeholders) 
    cur.execute(sql2, list(valuest.values()))
    try:
        valuest2 = {'id': values['id'], 'text': unify.open_text(open(os.path.join(os.path.join(tagged,folder), str(values['filepath'])+".txt"),'r', encoding='utf8').read())}
        columns = ', '.join(valuest2.keys())
        placeholders = ', '.join('?' * len(valuest2))
        sql3 = 'INSERT INTO tagged_texts ({}) VALUES ({})'.format(columns, placeholders) 
        cur.execute(sql3, list(valuest2.values()))
    except:
        valuest2 = {'id': values['id'], 'text': ""}
        columns = ', '.join(valuest2.keys())
        placeholders = ', '.join('?' * len(valuest2))
        sql3 = 'INSERT INTO tagged_texts ({}) VALUES ({})'.format(columns, placeholders) 
        cur.execute(sql3, list(valuest2.values()))
    con.commit()

  1%|▏         | 276/19010 [02:08<2:27:44,  2.11it/s]